# Pipelines do SkLearn

Hoje vamos aprender uma ferramenta poderosíssima do `sklearn`: _pipelines_ (que em tradução literal é _oleoduto_, mas eu prefiro algo como _linha de montagem_). 

As pipelines nos permitem colocar em sequência todos o passos do nosso projeto de Machine Learning e também automatizar o pré-processamento, treinamento e afinação (_tuning_) de hiperparâmetros. Além disso, elas nos permitem fazer um `GridSearch` não só nos hyperparâmetros de um determinado modelo, mas também nos parâmetros que usamos no pré-processamento.

- _Será que eu preencho os valores nulos com a média ou mediana?_
- _Será que eu uso ou não essa determinada feature?_

Essas entre outras perguntas são muito comuns quando estamos lidando com um projeto. As pipelines permitem que a gente ache exatamente qual é o melhor score que nossos modelos podem ter lidando com essas perguntas através de parâmetros que passamos para um `GridSearch`.

# Importantando bibliotecas e dados

Primeiro, como sempre, vamos importar as bibliotecas e dados que vamos utilizar.

Esse tutorial foi inspirado no segundo capítulo do livro _Hands-On Machine Learning_, o notebook do capítulo pode ser encontrado no [GitHub](https://github.com/ageron/handson-ml2). Então vamos importar o data set utilizado por ele.

In [1]:
import pandas as pd
import numpy as np

Contextualizando, temos dados de casas no estado da California e seus preços, que é o que queremos predizer.

In [2]:
input_path = 'housing.csv'
df = pd.read_csv(input_path)
df.sample(7)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
7027,-118.09,33.96,36.0,1116.0,229.0,719.0,233.0,3.4250,163200.0,<1H OCEAN
17785,-121.84,37.37,28.0,1579.0,339.0,1252.0,353.0,4.1615,214800.0,<1H OCEAN
20632,-121.45,39.26,15.0,2319.0,416.0,1047.0,385.0,3.1250,115600.0,INLAND
18185,-122.04,37.38,38.0,2850.0,550.0,1518.0,514.0,4.2028,273600.0,<1H OCEAN
15773,-122.45,37.76,50.0,2518.0,507.0,979.0,516.0,4.6912,500001.0,NEAR BAY
10462,-117.64,33.48,12.0,2007.0,397.0,1033.0,373.0,5.6754,275900.0,<1H OCEAN
15345,-117.38,33.21,31.0,1502.0,367.0,1514.0,342.0,2.6442,103300.0,NEAR OCEAN


Vemos que existem alguns valores nulos na coluna `total_bedrooms`. 

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


# Pré-processamento sem pipelines

Antes de ver como usar pipelines, vamos demonstrar como faríamos o pré-processamento dos dados sem elas.
Os passos do pré-processamento serão:
- Preencher os valores nulos
- Normalizar e padronizar os valores numéricos
- Codificar os valores categóricos utilizando One-Hot Encoding

In [4]:
from sklearn.impute import SimpleImputer

def fill_na(df, strategy='median'):

    num_values = list(df.columns[:-1])

    imputer = SimpleImputer(strategy=strategy)
    imputer.fit(df[num_values])
    df[num_values] = imputer.transform(df[num_values])
    return df


df = fill_na(df)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20640 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


In [5]:
from sklearn.preprocessing import StandardScaler

def scale(df):
    num_values = list(df.columns[:-1])

    scaler = StandardScaler()
    scaler.fit(df[num_values])
    df[num_values] = scaler.transform(df[num_values])
    return df


df = scale(df)
df.sample(5)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
19341,-1.647279,1.399006,1.856182,-0.514202,-0.450279,-0.581470,-0.380675,-0.884510,-0.406055,<1H OCEAN
1116,-1.018374,1.937421,-0.289187,-0.469280,-0.533735,-0.519656,-0.514070,-0.477778,-1.017011,INLAND
12698,-0.928531,1.394324,0.346478,-0.158952,0.093378,-0.287413,0.009051,-0.895512,-0.963282,INLAND
10342,0.938218,-0.857653,-1.878348,0.248554,0.224524,0.036667,0.213068,0.728206,0.262098,<1H OCEAN
16467,-0.848670,1.169595,0.823227,-0.009976,0.231677,0.385473,0.404007,-1.024790,-1.179933,INLAND


In [6]:
from sklearn.preprocessing import OneHotEncoder

def encode(df):
    cat_values = ['ocean_proximity']
    
    encoder = OneHotEncoder()
    encoder.fit(df[cat_values])
    columns = [cat_values[0] + '_' + cat_name for cat_name in encoder.categories_][0]
    encoded = pd.DataFrame(encoder.transform(df[cat_values]).toarray(), columns=columns)
    return pd.concat([df, encoded.astype(int)], axis=1).drop('ocean_proximity', axis=1)


df = encode(df)
df

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity_<1H OCEAN,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY,ocean_proximity_NEAR OCEAN
0,-1.327835,1.052548,0.982143,-0.804819,-0.972476,-0.974429,-0.977033,2.344766,2.129631,0,0,0,1,0
1,-1.322844,1.043185,-0.607019,2.045890,1.357143,0.861439,1.669961,2.332238,1.314156,0,0,0,1,0
2,-1.332827,1.038503,1.856182,-0.535746,-0.827024,-0.820777,-0.843637,1.782699,1.258693,0,0,0,1,0
3,-1.337818,1.038503,1.856182,-0.624215,-0.719723,-0.766028,-0.733781,0.932968,1.165100,0,0,0,1,0
4,-1.337818,1.038503,1.856182,-0.462404,-0.612423,-0.759847,-0.629157,-0.012881,1.172900,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20635,-0.758826,1.801647,-0.289187,-0.444985,-0.388283,-0.512592,-0.443449,-1.216128,-1.115804,0,1,0,0,0
20636,-0.818722,1.806329,-0.845393,-0.888704,-0.922403,-0.944405,-1.008420,-0.691593,-1.124470,0,1,0,0,0
20637,-0.823713,1.778237,-0.924851,-0.174995,-0.123608,-0.369537,-0.174042,-1.142593,-0.992746,0,1,0,0,0
20638,-0.873626,1.778237,-0.845393,-0.355600,-0.304827,-0.604429,-0.393753,-1.054583,-1.058608,0,1,0,0,0


Então, basicamente criei três funções, uma para cada tarefa e vamos modificando o `DataFrame` original até ele se tornar a matriz `X` que vamos passar para nosso modelo de Machine Learning (usando `.fit`).

# Pipelines

Agora vejamos como fazer o mesmo processo, mas utilizando Pipelines. 

Para criar um `Pipeline`, importamos esse objeto e o instanciamos passando uma lista de tuplas. 
- O primeiro valor da tupla é o nome daquele passo na nossa linha de montagem;
- O segundo valor é um `Transformer` do `sklearn`. Ou seja, deve ser um objeto que possua os métodos `transform` e `fit` (pelo menos).

Vamos falar mais em detalhes de `Transformers` mais para frente.

**_Nota_**: o mais correto é dizer que todos os passos devem ser `Transformers`, exceto o último, que pode ser um `Estimator` (ou seja, possuir o método `predict`).

Após criar esse `Pipeline`, podemos chamar os seus métodos `fit` e `transform`. Basicamente, o que ele faz é chamar em sequência cada `Transformer` passando para o próximo a saída retornada pelo anterior. 

In [7]:
from sklearn.pipeline import Pipeline

num_pipeline = Pipeline([
     ('fillna', SimpleImputer(strategy='median')),
     ('scaler', StandardScaler()),
])

num_values = list(df.columns[:9])
num_pipeline.fit(df[num_values])
num_df_transformed = num_pipeline.transform(df[num_values])
num_df_transformed

array([[-1.32783522,  1.05254828,  0.98214266, ..., -0.97703285,
         2.34476576,  2.12963148],
       [-1.32284391,  1.04318455, -0.60701891, ...,  1.66996103,
         2.33223796,  1.31415614],
       [-1.33282653,  1.03850269,  1.85618152, ..., -0.84363692,
         1.7826994 ,  1.25869341],
       ...,
       [-0.8237132 ,  1.77823747, -0.92485123, ..., -0.17404163,
        -1.14259331, -0.99274649],
       [-0.87362627,  1.77823747, -0.84539315, ..., -0.39375258,
        -1.05458292, -1.05860847],
       [-0.83369581,  1.75014627, -1.00430931, ...,  0.07967221,
        -0.78012947, -1.01787803]])

Compare com o `DataFrame` que geramos pelo pré-processamento anteriormente e você verá que o resultado é o mesmo. A diferença é que aqui temos diretamente um `ndarray` do `numpy` ao invés de um `DataFrame` do `pandas` (na prática, não muda muito, pois o método `fit` de um estimador transforma internamente `DataFrames` em `ndarrays`).

# Transformações diferentes para colunas diferentes

Bem, agora precisamos fazer o One-Hot Encoding na coluna `ocean_proximity` e juntar com essa matriz que o `Pipeline` gerou, certo?

Infelizmente, apenas com `Pipelines` isso não é possível, pois o `Pipeline` não considera as diferentes colunas de uma matriz. Ele apenas aplica as transformações (é por isso que na hora de dar fit e transform, eu chamo `df[num_values]` ao invés de `df` inteiro).

Entretanto, o objeto `ColumnTransformer` nos permite fazer exatamente o que precisamos: transformar um conjunto de colunas de um jeito e um outro conjunto de colunas de outro e juntar essas colunas em uma única matriz.

Eles funcionam muito similar a `Pipelines`, a diferença é que a tupla recebe um valor a mais: as colunas em que aquela transformação será aplicada.

In [8]:
from sklearn.compose import ColumnTransformer

cat_values = ['ocean_proximity']

full_pipeline = ColumnTransformer([
    ('numeric', num_pipeline, num_values), 
    ('categorical', OneHotEncoder(), cat_values)
])

df = pd.read_csv(input_path)
X = full_pipeline.fit_transform(df)
X

array([[-1.32783522,  1.05254828,  0.98214266, ...,  0.        ,
         1.        ,  0.        ],
       [-1.32284391,  1.04318455, -0.60701891, ...,  0.        ,
         1.        ,  0.        ],
       [-1.33282653,  1.03850269,  1.85618152, ...,  0.        ,
         1.        ,  0.        ],
       ...,
       [-0.8237132 ,  1.77823747, -0.92485123, ...,  0.        ,
         0.        ,  0.        ],
       [-0.87362627,  1.77823747, -0.84539315, ...,  0.        ,
         0.        ,  0.        ],
       [-0.83369581,  1.75014627, -1.00430931, ...,  0.        ,
         0.        ,  0.        ]])

Note que o objeto `Pipeline` também é um `Transformer` (pois tem os métodos `fit` e `transform`), então podemos considerá-lo como se fosse uma caixa preta e passar para nosso `ColumnTransformer`.

E é isso, temos exatamente a mesma coisa que tínhamos feito antes, mas de uma forma muuito mais prática (claro, entender como `Pipelines` funcionam talvez não seja exatamente fácil, mas com certeza recompensa muito).

Esse tutorial poderia parar por aqui. Mas vamos ver algumas outras coisinhas que podemos fazer utilizando `Pipelines`.

# Criando seus próprios Transformers

O sklearn nos fornece vários transformadores nativos, mas podemos querer criar outros. Um exemplo muito prático é fazer um `Transformer` que gere novas features do seu processo de Feature Engineering. Outro exemplo seria um `Transformer` que matém apenas determinadas features do Data Set (Feature Selection).

Vamos mostrar uma mistura desses dois exemplos. Vou criar um `Transformer` que cria duas novas features e tem um parâmetro booleano se devemos criar ou não uma terceira nova feature (assim poderemos testar mais para frente se incluir ou não incluir essa feature é melhor ou não). 

Para criar um `Transformer`, basicamente basta criar uma classe que implemente os métodos `fit` e `transform`, como já falei (sim, não precisa herdar de nenhuma classe do `sklearn` com o nome `Transformer` ou coisa do tipo. Mostrei isso no Apêndice A do notebook).

Entretanto, existem duas classes que costumamos herdar, pois elas nos ajudam:
- `TransformerMixin` cria para nós um método `fit_transform` automaticamente usando nossos método `fit` e `transform`; e
- `BaseEstimator` cria para nós os métodos `set_params` e `get_params` que são utilizados pelo `GridSearchCV` internamente.

In [9]:
from sklearn.base import BaseEstimator, TransformerMixin

class CreateNewFeatures(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room=True):
        self.rooms_ix = 3
        self.bedrooms_ix = 4
        self.population_ix = 5
        self.households_ix = 6
        self.add_bedrooms_per_room = add_bedrooms_per_room
    
    def fit(self, X, y=None):
        return self # não fazemos nada
    
    def transform(self, X):
        rooms_per_household = X[:, self.rooms_ix] / X[:, self.households_ix]
        population_per_household = X[:, self.population_ix] / X[:, self.households_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, self.bedrooms_ix] / X[:, self.rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household, bedrooms_per_room]
        return np.c_[X, rooms_per_household, population_per_household]

new_features = CreateNewFeatures(add_bedrooms_per_room=True)
housing_extra_features = new_features.transform(df.values)
housing_extra_features

array([[-122.23, 37.88, 41.0, ..., 6.984126984126984, 2.5555555555555554,
        0.14659090909090908],
       [-122.22, 37.86, 21.0, ..., 6.238137082601054, 2.109841827768014,
        0.15579659106916466],
       [-122.24, 37.85, 52.0, ..., 8.288135593220339, 2.8022598870056497,
        0.12951601908657123],
       ...,
       [-121.22, 39.43, 17.0, ..., 5.20554272517321, 2.325635103926097,
        0.21517302573203195],
       [-121.32, 39.43, 18.0, ..., 5.329512893982808, 2.1232091690544412,
        0.21989247311827956],
       [-121.24, 39.37, 16.0, ..., 5.254716981132075, 2.616981132075472,
        0.22118491921005387]], dtype=object)

E pronto, temos um `Transformer` feito por nós mesmos. O limite do que se pode fazer é basicamente definido pela nossa imaginação =)

Agora podemos utilizar esses `Transformers` na nossa `Pipeline` de antes para deixá-la mais sofisticada ainda.

In [10]:
num_pipeline = Pipeline([
     ('fillna', SimpleImputer(strategy='median')),
     ('feature_creator', CreateNewFeatures(add_bedrooms_per_room=True)),
     ('scaler', StandardScaler()),
])


df_transformed = num_pipeline.fit_transform(df[num_values])
df_transformed

array([[-1.32783522,  1.05254828,  0.98214266, ...,  0.62855945,
        -0.04959654, -1.02998783],
       [-1.32284391,  1.04318455, -0.60701891, ...,  0.32704136,
        -0.09251223, -0.8888972 ],
       [-1.33282653,  1.03850269,  1.85618152, ...,  1.15562047,
        -0.02584253, -1.29168566],
       ...,
       [-0.8237132 ,  1.77823747, -0.92485123, ..., -0.09031802,
        -0.0717345 ,  0.02113407],
       [-0.87362627,  1.77823747, -0.84539315, ..., -0.04021111,
        -0.09122515,  0.09346655],
       [-0.83369581,  1.75014627, -1.00430931, ..., -0.07044252,
        -0.04368215,  0.11327519]])

In [11]:
preprocessing_pipeline = ColumnTransformer([
    ('numeric', num_pipeline, num_values),
    ('categorical', OneHotEncoder(), cat_values)
])

df = pd.read_csv(input_path)
X = preprocessing_pipeline.fit_transform(df)
X

array([[-1.32783522,  1.05254828,  0.98214266, ...,  0.        ,
         1.        ,  0.        ],
       [-1.32284391,  1.04318455, -0.60701891, ...,  0.        ,
         1.        ,  0.        ],
       [-1.33282653,  1.03850269,  1.85618152, ...,  0.        ,
         1.        ,  0.        ],
       ...,
       [-0.8237132 ,  1.77823747, -0.92485123, ...,  0.        ,
         0.        ,  0.        ],
       [-0.87362627,  1.77823747, -0.84539315, ...,  0.        ,
         0.        ,  0.        ],
       [-0.83369581,  1.75014627, -1.00430931, ...,  0.        ,
         0.        ,  0.        ]])

E _voilà_, temos novamente uma matriz prontinha para qualquer modelo de Machine Learning utilizar.

# Pipelines e Hyperparameter Tuning

Por fim, uma das funcionalidade que eu mais acho incrível dos `Pipelines` é a capacidade usá-los junto do `GridSearchCV` (ou `RandomizedSearchCV`) e podermos passar parâmetros dos transformadores em si para serem testados.

Vamos dar uma olhada:

In [12]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge

# Dividindo o DataFrame em matrizes X (de features) e y (o target)
# Além disso, definimos quais colunas são numéricas e quais são categóricas (para usar no ColumnTransformer)
target = 'median_house_value'
X = df.loc[:, df.columns != target]
y = df[target]
num_values = np.delete(X.columns, np.where(X.columns == 'ocean_proximity'))
cat_values = ['ocean_proximity']

# Agora definimos nossas Pipelines
num_pipeline = Pipeline([
     ('fillna', SimpleImputer(strategy='median')),
     ('feature_creator', CreateNewFeatures(add_bedrooms_per_room=True)),
     ('scaler', StandardScaler()),
])
preprocessing_pipeline = ColumnTransformer([
    ('numeric', num_pipeline, num_values),
    ('categorical', OneHotEncoder(categories=[df['ocean_proximity'].unique()]), cat_values)
])
final_pipe = Pipeline([
    ('preprocessing', preprocessing_pipeline),
    ('ridge', Ridge())
])

# Veja que a última Pipeline tem dois passos: o pré-processamento e o estimador (o Ridge, que é um modelo linear)

# Agora tem uma parte que pode ser um pouco complexa, mas basta enteder o que está dentro de o que
# Na hora de passar os nomes dos parâmetros para o GridSearch, ele utiliza os nomes que passamos nas tuplas 
# o utiliza dois underlines para se referir a um parâmetro daquele objeto (seria análogo ao ponto que usamos normalmente)
params = {
    'preprocessing__numeric__feature_creator__add_bedrooms_per_room' : [False, True],
    'preprocessing__numeric__fillna__strategy': ['median', 'mean'],
    'ridge__alpha' : [0.1, 1, 10],
}

# De resto, é tudo igual, passamos o estimador (esse Pipeline é um estimador pois o último passo é o Ridge, que é um estimador)
# e passamos os parâmetros, além de outros parâmetros que normalmente usamos em um GridSearch
gs = GridSearchCV(final_pipe, params, cv=5, n_jobs=-1)
gs.fit(X, y)
gs.best_params_ # E temos os melhores parâmetros automaticamente =)

{'preprocessing__numeric__feature_creator__add_bedrooms_per_room': True,
 'preprocessing__numeric__fillna__strategy': 'median',
 'ridge__alpha': 10}

Na minha opinião, a parte mais difícil de entender é esses nomes enormes de parâmetros e entender as abstrações de um `Pipeline` (pensar que o `Pipeline` herda o tipo do objeto no último passo - um `Estimator` ou `Transformer`). Por isso, vou deixar uma imagem para ficar mais claro.

<img src="Arvore-de-Pipelines.jpg" alt="drawing" width="500"/>

Além disso, para saber quais são os parâmetros de um `Pipeline`, podemos utilizar o método `get_params`, que nos retorna um dicionário com cada atributo e seu valor padrão. Olhe o Apêndice B para ver o retorno desse método para o nosso `Pipeline` final.

# Apêndice A

In [13]:
class MyTransformer():
    def __init__(self):
        pass
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return np.ones_like(X)
        
df = pd.read_csv(input_path)

my_pipeline = Pipeline([
    ('imputer', SimpleImputer()),
    ('test', MyTransformer()), 
])

my_pipeline.fit_transform(df[num_values])
# Mesmo não herdando de nada, o sklearn reconhece nossa classe como um Transformer =)

array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]])

# Apêndice B

In [14]:
final_pipe.get_params()

{'memory': None,
 'steps': [('preprocessing',
   ColumnTransformer(transformers=[('numeric',
                                    Pipeline(steps=[('fillna',
                                                     SimpleImputer(strategy='median')),
                                                    ('feature_creator',
                                                     CreateNewFeatures()),
                                                    ('scaler', StandardScaler())]),
                                    Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
          'total_bedrooms', 'population', 'households', 'median_income'],
         dtype='object')),
                                   ('categorical',
                                    OneHotEncoder(categories=[array(['NEAR BAY', '<1H OCEAN', 'INLAND', 'NEAR OCEAN', 'ISLAND'],
         dtype=object)]),
                                    ['ocean_proximity'])])),
  ('ridge', Ridge())],
 'verbose': False,
 'preproce

# Conclusão e Aprofundamento

Por hoje é tudo =)

Espero que você tenha gostado e aproveitado bastante. 

Concluímos que os `Pipelines` são uma ferramenta incrível que podemos utilizar para agilizar muito nossa vida como cientistas de dados. Além disso, aprendemos um pouco sobre como o `sklearn` opera (os diferentes tipos de objetos dele) e como criar nossos próprios `Transformers`, além de aplicar diferentes transformações a diferentes colunas utilizando `ColumnTransformers`.

Para se aprofundar e revisar, recomendo dar uma buscada no livro de onde eu tirei esses exemplos, que já citei, além de dar uma olhada nos seguintes recursos:

[Livro - Hands On Machine Learning](https://www.oreilly.com/library/view/hands-on-machine-learning/9781492032632/)

[Guia do usuário - Pipeline and composite estimators](https://scikit-learn.org/stable/modules/compose.html#combining-estimators)

[Documentação Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html)

[Documentação make_pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.make_pipeline.html#sklearn.pipeline.make_pipeline)

[Documentação ColumnTransformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html#sklearn.compose.ColumnTransformer)

[Documentação TransformedTargetRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.compose.TransformedTargetRegressor.html#sklearn.compose.TransformedTargetRegressor)

[Tutorial do towards data science](https://towardsdatascience.com/pipelines-custom-transformers-in-scikit-learn-the-step-by-step-guide-with-python-code-4a7d9b068156)



**~Lucas Paiolla, 23/04/2021**